In [ ]:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
import math
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
from pytvfemd import tvfemd

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = [], []
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
def prepare_input_data(scaled, n_in, n_out=144, n_vars=4, train_proportion=0.8):    
    level = scaled[:, 0]
    q = scaled[:, 1]
    level_tvfemd = tvfemd(level)
    q_tvfemd = tvfemd(q)    
    level_q_tvfemd = np.hstack((level_tvfemd, q_tvfemd))
    input_reframed = series_to_supervised(level_q_tvfemd, n_in, n_out)
    input_contain_vars = []
    for i in range(1, n_in+1):
        input_contain_vars += [('var%d(t-%d)' % (j, i)) for j in range(1,n_vars+1)]  
    input_data = input_reframed [ input_contain_vars + ['var1(t)'] + [('var1(t+%d)' % (j)) for j in range(1,n_out)]]
    input_col_names = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 
                       'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'X21', 'X22', 'X23']
    input_contain_vars = []
    for i in range(n_vars):
        input_contain_vars += [('%s(t-%d)' % (input_col_names[i], j)) for j in range(1,n_in+1)]  
    input_data.columns = input_contain_vars +  ['X0(t)'] + [('X0(t+%d)' % (j)) for j in range(1,n_out)]
    input_values = input_data.values
    input_n_train = round(input_data.shape[0]*train_proportion)
    input_train = input_values[:input_n_train, :]
    input_test = input_values[input_n_train:, :]
    input_train_X, input_train_y = input_train[:, :n_in*n_vars], input_train[:, n_in*n_vars:]
    input_test_X, input_test_y = input_test[:, :n_in*n_vars], input_test[:, n_in*n_vars:]
    input_train_X = input_train_X.reshape((input_train_X.shape[0], n_in, n_vars))
    input_test_X = input_test_X.reshape((input_test_X.shape[0], n_in, n_vars))
    
    return input_train_X, input_test_X

In [ ]:
def prepare_output_data(scaled, n_in, n_out=144, n_vars=4, train_proportion=0.8):
    reframed = series_to_supervised(scaled, n_in, n_out)
    contain_vars = []
    for i in range(1, n_in+1):
        contain_vars += [('var%d(t-%d)' % (j, i)) for j in range(1,n_vars+1)]  
    data = reframed [ contain_vars + ['var1(t)'] + [('var1(t+%d)' % (j)) for j in range(1,n_out)]]
    col_names = ['Y', 'X1', 'X2', 'X3']
    contain_vars = []
    for i in range(n_vars):
        contain_vars += [('%s(t-%d)' % (col_names[i], j)) for j in range(1,n_in+1)]  
    data.columns = contain_vars +  ['Y(t)'] + [('Y(t+%d)' % (j)) for j in range(1,n_out)]
    values = data.values
    n_train = round(data.shape[0]*train_proportion)
    train = values[:n_train, :]
    test = values[n_train:, :]
    train_X, train_y = train[:, :n_in*n_vars], train[:, n_in*n_vars:]
    test_X, test_y = test[:, :n_in*n_vars], test[:, n_in*n_vars:]
    train_X = train_X.reshape((train_X.shape[0], n_in, n_vars))
    test_X = test_X.reshape((test_X.shape[0], n_in, n_vars))
    
    return train_y, test_y

In [ ]:
def read_data(filepath):
    dataset = read_csv(filepath, encoding='utf-8', index_col=0)
    values = dataset.values
    values = values.astype('float32')
       
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)
    return scaler, scaled

In [ ]:
Dataset = read_csv(r'D:\Liuzhan\Data.csv', encoding='utf-8', index_col=0)
Values = Dataset.values
Values = Values.astype('float32')       
Scaler = MinMaxScaler(feature_range=(0, 1))
Scaled = Scaler.fit_transform(Values)

Level = Scaled[:, 0]
Q = Scaled[:, 1]
Level_tvfemd = tvfemd(Level)
Q_tvfemd = tvfemd(Q)    
Level_Q_tvfemd = np.hstack((Level_tvfemd, Q_tvfemd))
np.savetxt(r'D:\Liuzhan\TVFEMD Result.csv', Level_Q_tvfemd, delimiter=",")

In [ ]:
filepath = r'D:\Liuzhan\Data.csv'
n_in = 28
n_out = 4
n_vars_No_D = 2
n_vars_D = 24
inv_yhat_list = []
inv_y_list = []
validation = 0.8
scaler, scaled = read_data(filepath)
train_X, test_X = prepare_input_data(scaled, n_in, n_out, n_vars = n_vars_D, train_proportion=validation) 
train_Y, test_Y = prepare_output_data(scaled, n_in, n_out, n_vars = n_vars_No_D, train_proportion=validation)

print('type(train_X) = ', type(train_X))
print('train_X.shape = ', train_X.shape)
print('type(train_Y) = ', type(train_Y))
print('train_Y.shape = ', train_Y.shape)
print('type(test_X) = ', type(test_X))
print('test_X.shape = ', test_X.shape)
print('type(test_Y) = ', type(test_Y))
print('test_Y.shape = ', test_Y.shape)

In [ ]:
train_X_2D = train_X.reshape((train_X.shape[0], n_in*n_vars_D))
test_X_2D = test_X.reshape((test_X.shape[0], n_in*n_vars_D))
np.savetxt(r'D:\Liuzhan\train_X.csv', train_X_2D, delimiter=",")
np.savetxt(r'D:\Liuzhan\train_Y.csv', train_Y, delimiter=",")  
np.savetxt(r'D:\Liuzhan\test_X.csv', test_X_2D, delimiter=",") 
np.savetxt(r'D:\Liuzhan\test_Y.csv', test_Y, delimiter=",")

In [ ]:
import torch
import torch.nn as nn
from kan import KAN
from torch.nn import init

In [ ]:
train_X = torch.from_numpy(train_X).type(torch.Tensor)
test_X = torch.from_numpy(test_X).type(torch.Tensor)
train_Y = torch.from_numpy(train_Y).type(torch.Tensor)
test_Y = torch.from_numpy(test_Y).type(torch.Tensor)

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
class TVFEMD_EA_CNN_LSTM_KAN(nn.Module):
    hidden_cell_num = []
    def __init__(self, In_Channels, Out_Channels, Input_Size, Output_Size, Batch_Size, dropout, S, cell_num):
        super().__init__()
        self.output_size = Output_Size
        self.num_directions = 1
        self.batch_size = Batch_Size
        self.hidden_cell_num = cell_num

        self.mk=nn.Linear(Input_Size,S,bias=False)
        self.softmax=nn.Softmax(dim=2)  
        self.mv=nn.Linear(S,Input_Size,bias=False)        
        self.init_weights()
        self.relu = nn.ReLU(inplace=True)

        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=In_Channels, out_channels=Out_Channels, kernel_size=3),
            nn.ReLU(),
            nn.Conv1d(in_channels=Out_Channels,  
                      out_channels=Out_Channels, 
                      kernel_size=3),   
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=3, stride=1)            
        )
        
        self.lstm0 = nn.LSTMCell(Out_Channels, hidden_size=self.hidden_cell_num[0])
        self.dropout0 = nn.Dropout(p=dropout)
        self.lstm1 = nn.LSTMCell(input_size=self.hidden_cell_num[0], hidden_size=self.hidden_cell_num[1])  
        self.dropout1 = nn.Dropout(p=dropout)
        self.lstm2 = nn.LSTMCell(input_size=self.hidden_cell_num[1], hidden_size=self.hidden_cell_num[2])  
        self.dropout2 = nn.Dropout(p=dropout)
        self.lstm3 = nn.LSTMCell(input_size=self.hidden_cell_num[2], hidden_size=self.hidden_cell_num[3]) 
        self.dropout3 = nn.Dropout(p=dropout)
        self.lstm4 = nn.LSTMCell(input_size=self.hidden_cell_num[3], hidden_size=self.hidden_cell_num[4])  
        self.dropout4 = nn.Dropout(p=dropout)
        
        self.kan=KAN(width=[self.hidden_cell_num[4], 1, 1, 3, Output_Size], grid=20, k=3).to(device)

    def forward(self, input_seq):
        batch_size, seq_len = input_seq.shape[0], input_seq.shape[1]
        # batch_size, hidden_size
        h_l0 = torch.zeros(batch_size, self.hidden_cell_num[0]).to(device)
        c_l0 = torch.zeros(batch_size, self.hidden_cell_num[0]).to(device)
        h_l1 = torch.zeros(batch_size, self.hidden_cell_num[1]).to(device)
        c_l1 = torch.zeros(batch_size, self.hidden_cell_num[1]).to(device)
        h_l2 = torch.zeros(batch_size, self.hidden_cell_num[2]).to(device)
        c_l2 = torch.zeros(batch_size, self.hidden_cell_num[2]).to(device)
        h_l3 = torch.zeros(batch_size, self.hidden_cell_num[3]).to(device)
        c_l3 = torch.zeros(batch_size, self.hidden_cell_num[3]).to(device)
        h_l4 = torch.zeros(batch_size, self.hidden_cell_num[4]).to(device)
        c_l4 = torch.zeros(batch_size, self.hidden_cell_num[4]).to(device)

        attn = self.mk(input_seq) 
        attn = self.softmax(attn) 
        attn = attn/torch.sum(attn,dim=1,keepdim=True) 
        out = self.mv(attn) 
        x = input_seq + out
        
        conv = x.permute(0, 2, 1)
        conv = self.conv(conv)
        conv = conv.permute(0, 2, 1)
        conv = self.relu(conv)
        
        output = []
        for t in range(conv.shape[1]):
            h_l0, c_l0 = self.lstm0(conv[:, t, :], (h_l0, c_l0))
            h_l0, c_l0 = self.dropout0(h_l0), self.dropout0(c_l0)
            h_l1, c_l1 = self.lstm1(h_l0, (h_l1, c_l1))
            h_l1, c_l1 = self.dropout1(h_l1), self.dropout1(c_l1)            
            h_l2, c_l2 = self.lstm2(h_l1, (h_l2, c_l2))
            h_l2, c_l2 = self.dropout2(h_l2), self.dropout2(c_l2)
            h_l3, c_l3 = self.lstm3(h_l2, (h_l3, c_l3))
            h_l3, c_l3 = self.dropout3(h_l3), self.dropout3(c_l3)
            h_l4, c_l4 = self.lstm3(h_l3, (h_l4, c_l4))
            h_l4, c_l4 = self.dropout3(h_l4), self.dropout3(c_l4)
            output.append(h_l4)

        pred = self.kan(output[-1])

        return pred

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)     

In [ ]:
cell_num = [700, 700, 600, 600, 600]  
model = TVFEMD_EA_CNN_LSTM_KAN(In_Channels=n_vars_D, Out_Channels=64, Input_Size = n_vars_D, Output_Size = n_out, Batch_Size=1, dropout=0.3, S=64, cell_num = cell_num)

In [ ]:
model = model.to(device)
train_X = train_X.to(device)
test_X = test_X.to(device)
train_Y = train_Y.to(device)
test_Y = test_Y.to(device)

In [ ]:
import time
import os
import datetime


n_epochs = 10000
learning_rate = 0.001
epsilon = 0.0001
criterion = torch.nn.MSELoss(reduction='mean') # mean value of loss on all samples
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate, eps=epsilon)

loss_hist = np.zeros(n_epochs)
val_loss_hist = np.zeros(n_epochs)
start_time = time.time()
print("Start training time： ", datetime.datetime.now())

for t in range(n_epochs):
    y_train_pred = model(train_X)
    y_test_pred = model(test_X)
    loss = criterion(y_train_pred, train_Y)
    val_loss = criterion(y_test_pred, test_Y)    
    loss_hist[t] = loss.item()
    val_loss_hist[t] = val_loss.item()
    print("Epoch ", t, "/", n_epochs, " train MSE: ", loss.item(), " ;  test MSE: ", val_loss.item())
    optimiser.zero_grad() 
    loss.backward()       
    optimiser.step()      
    torch.cuda.empty_cache() 
    
training_time = time.time() - start_time
print("Training time: {}".format(training_time))

csv_path = r'D:\Liuzhan\training process.csv'
np.savetxt(csv_path, np.column_stack((loss_hist, val_loss_hist)), delimiter=",", header="Training loss,Test loss", comments='')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.figure(figsize=(5,4),dpi=400)  

plt.plot(loss_hist, color='blue', label='train')
plt.plot(val_loss_hist, color='red', label='test')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend(["Train loss", "Test loss"], ncol=1, prop={"size": 10})  # "family": "Times New Roman", 
plt.title('MSE - Epoch')
plt.show()

In [ ]:
from statistics import mean

print('type(test_X) = ', type(test_X))
print('test_X.shape = ', test_X.shape)
yhat = model(test_X)
print('type(yhat) = ', type(yhat))
print('yhat.shape = ', yhat.shape)

inv_yhat_list = []
inv_y_list = []
inv_y = [] 
inv_yhat = []

scale_new = MinMaxScaler()
scale_new.min_, scale_new.scale_ = scaler.min_[0], scaler.scale_[0] 
inv_yhat = scale_new.inverse_transform(yhat.detach().numpy()) 
inv_y = scale_new.inverse_transform(test_Y.detach().numpy()) 
inv_yhat_list.append(inv_yhat)
inv_y_list.append(inv_y)

print('type(inv_yhat) = ', type(inv_yhat))
print('inv_yhat.shape = ', inv_yhat.shape)
print('type(inv_y) = ', type(inv_y))
print('inv_y.shape = ', inv_y.shape)
csv_path = r'D:\Liuzhan\Actual.csv'
np.savetxt(csv_path, inv_y, delimiter=",", header="Actual", comments='')
csv_path = r'D:\Liuzhan\Predicted.csv'
np.savetxt(csv_path, inv_yhat, delimiter=",", header="Predicted", comments='')

MSE_level = []
RMSE_level = []
MAPE_level = []
MAE_level = []
   
for j in range(inv_yhat.shape[0]):
    mse_level = mean_squared_error(inv_y[j], inv_yhat[j])
    rmse_level = sqrt(mean_squared_error(inv_y[j], inv_yhat[j]))
    mape_level = mean_absolute_percentage_error(inv_y[j], inv_yhat[j])
    mae_level = mean_absolute_error(inv_y[j], inv_yhat[j])
        
    MSE_level.append(mse_level)
    RMSE_level.append(rmse_level)
    MAPE_level.append(mape_level)
    MAE_level.append(mae_level)
    
print('')
print('MSE = ', mean(MSE_level)) 
print('RMSE = ', mean(RMSE_level))
print('MAPE = ', mean(MAPE_level))
print('MAE = ', mean(MAE_level))